In [192]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.datasets import load_breast_cancer
from torchimize.functions import lsq_lma

In [193]:
device = torch.device('cpu')
# For data preprocessing. To be replcaed with AR section.
cancer_data = pd.read_csv('/Users/waldo/Documents/23 HT/Data Mining 1DL370/Assignment4/cancer.csv',sep=';')
cancer_data.iloc[cancer_data['diagnosis'] == 'M',1] = 1
cancer_data.iloc[cancer_data['diagnosis'] == 'B',1] = 0
cancer_benign = cancer_data[cancer_data['diagnosis'] == 0]

X, y = cancer_data.iloc[:,2:], cancer_data.iloc[:,1]
X_normed = pd.DataFrame(MinMaxScaler().fit_transform(X))
X = X_normed.astype('float')
y = y.astype('float')
X = X.values

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state=0)

X_train = torch.tensor(X_train, dtype=torch.float32).to(device)
y_train = torch.tensor(y_train, dtype=torch.float32).to(device)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test.to_numpy(), dtype=torch.float32)

In [194]:
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.input_layer = nn.Linear(32, 11)  # 32 input neurons to 11 hidden neurons. This will be changed to actual number of features.
        self.hidden_layer = nn.Linear(11, 1)  # 11 hidden neurons to 1 output neuron
    
    def forward(self, x):
        x = torch.tanh(self.input_layer(x))  # Apply tanh activation to the input layer
        x = torch.sigmoid(self.hidden_layer(x))  # Apply sigmoid activation to the output layer for binary classification
        return x

In [195]:
# Initialize the model
model = MLP()
model = model.to(device)

criterion = nn.BCELoss()

# Define the optimizer (Adam optimizer). To be changed to Lev-Mar
optimizer = optim.Adam(model.parameters(), lr=0.1)


In [196]:
num_epochs = 44

for epoch in range(num_epochs):
    outputs = model(X_train)
    loss = criterion(outputs, y_train.view(-1, 1))
    # Backward pass and optimization
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()  # Update weights
    
    #if (epoch + 1) % 1000 == 0:
       # print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')
print("Training completed")


Training completed


In [197]:
# Evaluate the model
with torch.no_grad():
    model.eval()
    predictions = model(X_test)
    predictions = (predictions > 0.5).float()
    accuracy = torch.sum(predictions.view(-1) == y_test).item() / y_test.size(0)
    print(f'Accuracy on test data: {accuracy:.4f}')


Accuracy on test data: 0.9737
